# Vorhersage Industrieller Daten

## 1. Einleitung

### Problemstellung
In vielen technischen und industriellen Anwendungen ist es notwendig, zeitabhängige Kennzahlen zuverlässig vorherzusagen. Typische Beispiele sind:
- **Energieverbrauch** von Maschinen oder Gebäuden  
- **Durchsatz** in Produktionsanlagen  
- **Ausfallraten** oder Wartungsbedarfe von Systemen  

Diese Größen liegen meist als **Zeitreihen** vor und unterliegen Trends, saisonalen Effekten sowie zufälligen Schwankungen.

### Was ist Forecasting?
Forecasting (Zeitreihenprognose) bezeichnet die Vorhersage zukünftiger Werte auf Basis historischer Daten. Dabei werden statistische oder maschinelle Lernverfahren eingesetzt, um Muster in der Vergangenheit zu erkennen und in die Zukunft fortzuschreiben.

### Motivation
Kurz- bis mittelfristige Vorhersagen sind besonders wichtig, um:
- Ressourcen effizient zu planen (z. B. Energie, Personal),
- Ausfälle frühzeitig zu erkennen,
- Kosten zu reduzieren und
- fundierte Entscheidungen auf Basis von Daten zu treffen.

### Ziel des Notebooks
Ziel dieses Jupyter Notebooks ist es, **klassische Zeitreihenmodelle** mit **modernen Ansätzen** zu vergleichen:
- Klassisch: **ARIMA**
- Modern: **Prophet** und ein einfaches **LSTM-Netzwerk**

Die Modelle werden hinsichtlich Prognosequalität, Interpretierbarkeit und Implementierungsaufwand gegenübergestellt.


## 2. Datenbeschreibung

### Herkunft des Datensatzes
Der verwendete Datensatz stammt von **kaggle.com** und ist ein öffentlich verfügbarer **Beispieldatensatz**. Er beschreibt den **elektrischen Energieverbrauch einer Stahlfabrik** und wird häufig für Analysen im Bereich industrieller Zeitreihen und Energieprognosen eingesetzt.

### Enthaltene Variablen
Der Datensatz liegt im **CSV-Format** vor und enthält folgende Variablen: (siehe auch Später df.info() ausgabe)

- **date**: Zeitstempel der Messung  
- **Usage_kWh**: Elektrischer Energieverbrauch in Kilowattstunden (Zielgröße)  
- **Lagging_Current_Reactive.Power_kVarh**: Nachlaufende Blindleistung  
- **Leading_Current_Reactive_Power_kVarh**: Vorlaufende Blindleistung  
- **CO2(tCO2)**: Geschätzte CO₂-Emissionen  
- **Lagging_Current_Power_Factor**: Leistungsfaktor (nachlaufend)  
- **Leading_Current_Power_Factor**: Leistungsfaktor (vorlaufend)  
- **NSM**: Number of Seconds from Midnight (Sekunden seit Mitternacht)  
- **WeekStatus**: Kennzeichnung von Wochentag oder Wochenende  
- **Day_of_week**: Name des Wochentages  
- **Load_Type**: Art der elektrischen Last (z. B. *Light Load*)

### Beispielhafte Ansicht des Datensatzes
Ein Auszug aus den ersten Zeilen des Datensatzes ist unten dargestellt:

In [14]:
import pandas as pd

df = pd.read_csv("../data/steel_industry_data.csv")

df.head()

,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load


In [15]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35040 entries, 0 to 35039
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   date                                  35040 non-null  object 
 1   Usage_kWh                             35040 non-null  float64
 2   Lagging_Current_Reactive.Power_kVarh  35040 non-null  float64
 3   Leading_Current_Reactive_Power_kVarh  35040 non-null  float64
 4   CO2(tCO2)                             35040 non-null  float64
 5   Lagging_Current_Power_Factor          35040 non-null  float64
 6   Leading_Current_Power_Factor          35040 non-null  float64
 7   NSM                                   35040 non-null  int64  
 8   WeekStatus                            35040 non-null  object 
 9   Day_of_week                           35040 non-null  object 
 10  Load_Type                             35040 non-null  object 
dtypes: float64(6), 

### Zeitraum und zeitliche Auflösung
Die Daten beginnen am **01.01.2018** und liegen in einer **viertelstündlichen Auflösung (15 Minuten)** vor. Damit eignen sie sich sowohl für kurz- als auch mittelfristige Prognosen des Energieverbrauchs.

bei 35040 Datensatzeinträgen ergibt das ein Zeitraum von 365 Tagen.

In [16]:
SETS_PER_HOUR = 4
HOURS_PER_DAY = 24

print(int(len(df)/(SETS_PER_HOUR * HOURS_PER_DAY)))

365
